<a href="https://colab.research.google.com/github/dmainf/kaggle/blob/main/titanic_cope1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Perished 死亡したかどうか 0 = No, 1 = Yes

Pclass チケットのクラス 1 = 1st, 2 = 2nd, 3 = 3rd

Name 名前

Sex 性別

Age 年齢

SibSp 乗船していた兄弟姉妹・配偶者の数

Parch 乗船していた親・子供の数

Ticket チケット番号

Fare チケット料金

Cabin キャビン番号

embarked 乗船した港 C = Cherbourg, Q = Queenstown, S = Southampton

Perished 死亡したかどうか 0 = No, 1 = Yes

Pclass チケットのクラス 1 = 1st, 2 = 2nd, 3 = 3rd

Sex 性別

Age 年齢

SibSp 乗船していた兄弟姉妹・配偶者の数

Parch 乗船していた親・子供の数

Fare チケット料金

embarked 乗船した港 C = Cherbourg, Q = Queenstown, S = Southampton

In [9]:
from google.colab import drive
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold

n_epoc = 10
batch_size = 32

# 1. データ読み込み
drive.mount('/content/drive')
path = '/content/drive/MyDrive/GCI2025_colab/Competition01/data/'
train_df = pd.read_csv(path + 'train.csv')
test_df  = pd.read_csv(path + 'test.csv')

# 2. 欠損値処理
for df in (train_df, test_df):
  df['Age'] = df['Age'].fillna(df['Age'].mean())
  df['Fare'] = df['Fare'].fillna(df['Fare'].mean())
train_df['Embarked'] = train_df['Embarked'].fillna(train_df['Embarked'].mode()[0])
test_df ['Embarked'] = test_df ['Embarked'].fillna(train_df['Embarked'].mode()[0])

# 3. 不要列削除
drop_cols = ['PassengerId','Name','Ticket','Cabin']
train_df.drop(drop_cols, axis=1, inplace=True)
test_df.drop(drop_cols, axis=1, inplace=True)

# 4. ラベル分離
X = train_df.drop('Perished', axis=1)
y = train_df['Perished']
X_test = test_df.copy()

# 5. カテゴリ変数の one-hot
X = pd.get_dummies(X, columns=['Sex','Embarked'])
X_test = pd.get_dummies(X_test, columns=['Sex','Embarked'])
# train にあって test にない列があれば 0 補完
X_test = X_test.reindex(columns=X.columns, fill_value=0)

# 6. 標準化
scaler = StandardScaler()
X[X.columns]       = scaler.fit_transform(X[X.columns])
X_test[X.columns]  = scaler.transform(X_test[X.columns])

# 7. train/valid 分割
x_train, x_valid, y_train, y_valid = train_test_split(
    X.values, y.values, test_size=0.3, random_state=42
)

# 8. DataLoader 準備
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
def to_dataset(x, y=None):
    x = torch.from_numpy(x.astype(np.float32))
    if y is None:
        return torch.utils.data.TensorDataset(x)
    y = torch.from_numpy(y).long()
    return torch.utils.data.TensorDataset(x, y)

train_loader = torch.utils.data.DataLoader(
    to_dataset(x_train, y_train), batch_size, shuffle=True
)
valid_loader = torch.utils.data.DataLoader(
    to_dataset(x_valid, y_valid), batch_size, shuffle=False
)
test_loader  = torch.utils.data.DataLoader(
    to_dataset(X_test.values), batch_size, shuffle=False
)

# 9. モデル定義
class MLP(nn.Module):
    def __init__(self, in_dim, out_dim):
        super().__init__()
        self.net = nn.Sequential(
            # 入力 → 128
            nn.Linear(in_dim, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            # 128 → 64
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            # 64 → 出力
            nn.Linear(64, out_dim)
        )

    def forward(self, x):
        return self.net(x)


in_dim  = x_train.shape[1]
out_dim = 2
model   = MLP(in_dim, out_dim).to(device)

# 10. 損失関数・オプティマイザ
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)

# 11. 学習ループ
for epoch in range(n_epoc):
    model.train()
    total_loss, correct, total = 0, 0, 0
    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device)
        logits = model(xb)
        loss   = criterion(logits, yb)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * xb.size(0)
        preds = logits.argmax(dim=1)
        correct   += (preds == yb).sum().item()
        total     += xb.size(0)
    train_loss, train_acc = total_loss/total, correct/total

    model.eval()
    v_loss, v_correct, v_total = 0, 0, 0
    with torch.no_grad():
        for xb, yb in valid_loader:
            xb, yb = xb.to(device), yb.to(device)
            logits = model(xb)
            loss   = criterion(logits, yb)
            v_loss   += loss.item() * xb.size(0)
            v_correct+= (logits.argmax(1)==yb).sum().item()
            v_total  += xb.size(0)
    val_loss, val_acc = v_loss/v_total, v_correct/v_total

    print(f"Epoch {epoch:03d} | "
          f"train_loss: {train_loss:.4f}, accuracy: {train_acc:.4f} | "
          f"valid_loss: {val_loss:.4f}, accuracy: {val_acc:.4f}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Epoch 000 | train_loss: 0.6302, accuracy: 0.6870 | valid_loss: 0.5550, accuracy: 0.7612
Epoch 001 | train_loss: 0.5017, accuracy: 0.7929 | valid_loss: 0.4636, accuracy: 0.7910
Epoch 002 | train_loss: 0.4671, accuracy: 0.8026 | valid_loss: 0.4463, accuracy: 0.7910
Epoch 003 | train_loss: 0.4515, accuracy: 0.8010 | valid_loss: 0.4320, accuracy: 0.7985
Epoch 004 | train_loss: 0.4515, accuracy: 0.8026 | valid_loss: 0.4302, accuracy: 0.7948
Epoch 005 | train_loss: 0.4371, accuracy: 0.7994 | valid_loss: 0.4258, accuracy: 0.8060
Epoch 006 | train_loss: 0.4289, accuracy: 0.8074 | valid_loss: 0.4245, accuracy: 0.8022
Epoch 007 | train_loss: 0.4245, accuracy: 0.8042 | valid_loss: 0.4223, accuracy: 0.8060
Epoch 008 | train_loss: 0.4238, accuracy: 0.8042 | valid_loss: 0.4232, accuracy: 0.8060
Epoch 009 | train_loss: 0.4129, accuracy: 0.8266 | valid_loss: 0.4226, accuracy

In [2]:
path = "/content/drive/MyDrive/GCI2025_colab/Competition01/"
# 3. サブミッション用テンプレート読み込み
submission = pd.read_csv(path + 'gender_submission.csv')

# 4. Perished 列を置き換え
submission['Perished'] = all_preds
submission.to_csv(path + 'submission.csv', index=False)
submission.to_csv('submission.csv', index=False)
